In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from typing import List, Text

from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.trainer.executor import Executor
from tfx.dsl.components.base import executor_spec
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_artifacts_resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.local.local_dag_runner import LocalDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

In [3]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [4]:
_pipeline_name = 'sampling_credit_card'
_sampling_root = os.path.dirname(".")
_data_root = os.path.join(_sampling_root, 'data')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_module_file = os.path.join(_sampling_root, 'sampler_utils.py')
_serving_model_dir = os.path.join(_sampling_root, 'serving_model', _pipeline_name)
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [5]:
_beam_pipeline_args = [
    '--direct_running_mode=multi_processing',
    # 0 means auto-detect based on on the number of CPUs available
    # during execution time.
    '--direct_num_workers=0',
]

In [6]:
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:60296,xor_checksum:1627065270,sum_checksum:1627065270"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [7]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [8]:
schema_gen = SchemaGen(
  statistics=statistics_gen.outputs['statistics'],
  infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [9]:
example_validator = ExampleValidator(
  statistics=statistics_gen.outputs['statistics'],
  schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [10]:
from sampler.component import Sampler

sampler = Sampler(
  input_data=example_gen.outputs['examples'],
  label='Class',
)
context.run(sampler)

ExecutionResult(
    component_id: Sampler
    execution_id: 5
    outputs:
        output_data: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 5
        type_id: 5
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Sampler/output_data/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "output_data"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Sampler"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
transform = Transform(
  examples=sampler.outputs['output_data'],
  schema=schema_gen.outputs['schema'],
  module_file=_module_file)
context.run(transform)

ERROR:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'sampler_utils@/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/_wheels/tfx_user_code_Transform-0.0+9703b0dd752afdc1bb9f6c369279db543827e0b49e156c9c89e847fad7881d51-py3-none-any.whl', 'preprocessing_fn': None} 'preprocessing_fn'


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: /var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Transform/transform_graph/6/.temp_path/tftransform_tmp/83a2ef1bdb234b59a1ed40a460682866/assets
INFO:tensorflow:Assets written to: /var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Transform/transform_graph/6/.temp_path/tftransform_tmp/26a233c5d9a844f2a13ef4708e938959/assets


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 5
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 8
        type_id: 15
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [12]:
latest_model_resolver = resolver.Resolver(
  strategy_class=latest_artifacts_resolver.LatestArtifactsResolver,
  latest_model=Channel(type=Model)).with_id('latest_model_resolver')
context.run(latest_model_resolver)

ExecutionResult(
    component_id: latest_model_resolver
    execution_id: 7
    outputs:
        latest_model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
trainer = Trainer(
  module_file=_module_file,
  custom_executor_spec=executor_spec.ExecutorClassSpec(Executor),
  transformed_examples=transform.outputs['transformed_examples'],
  schema=schema_gen.outputs['schema'],
  base_model=latest_model_resolver.outputs['latest_model'],
  transform_graph=transform.outputs['transform_graph'],
  train_args=trainer_pb2.TrainArgs(num_steps=10000),
  eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)

ERROR:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 10000\n}', 'eval_args': '{\n  "num_steps": 5000\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'sampler_utils@/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/_wheels/tfx_user_code_Trainer-0.0+9703b0dd752afdc1bb9f6c369279db543827e0b49e156c9c89e847fad7881d51-py3-none-any.whl'} 'trainer_fn'


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Trainer/model_run/8/Format-Serving', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 999, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:tensorflow:global_step/sec: 230.065
INFO:tensorflow:loss = 0.0062840385, step = 3000 (0.435 sec)
INFO:tensorflow:global_step/sec: 539.561
INFO:tensorflow:loss = 0.0059459493, step = 3100 (0.185 sec)
INFO:tensorflow:global_step/sec: 420.618
INFO:tensorflow:loss = 0.005591561, step = 3200 (0.238 sec)
INFO:tensorflow:global_step/sec: 410.577
INFO:tensorflow:loss = 0.005004892, step = 3300 (0.244 sec)
INFO:tensorflow:global_step/sec: 411.494
INFO:tensorflow:loss = 0.0051590493, step = 3400 (0.243 sec)
INFO:tensorflow:global_step/sec: 416.318
INFO:tensorflow:loss = 0.00491016, step = 3500 (0.240 sec)
INFO:tensorflow:global_step/sec: 416.089
INFO:tensorflow:loss = 0.004705638, step = 3600 (0.240 sec)
INFO:tensorflow:global_step/sec: 417.687
INFO:tensorflow:loss = 0.0047141155, step = 3700 (0.240 sec)
INFO:tensorflow:global_step/sec: 408.747
INFO:tensorflow:loss = 0.0041478286, step = 3800 (0.245 sec)
INFO:

INFO:tensorflow:loss = 0.0011395914, step = 8500 (0.379 sec)
INFO:tensorflow:global_step/sec: 260.971
INFO:tensorflow:loss = 0.0011518754, step = 8600 (0.383 sec)
INFO:tensorflow:global_step/sec: 253.931
INFO:tensorflow:loss = 0.0011493787, step = 8700 (0.394 sec)
INFO:tensorflow:global_step/sec: 267.631
INFO:tensorflow:loss = 0.0011291124, step = 8800 (0.373 sec)
INFO:tensorflow:global_step/sec: 283.221
INFO:tensorflow:loss = 0.0012488792, step = 8900 (0.353 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 8991...
INFO:tensorflow:Saving checkpoints for 8991 into /var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Trainer/model_run/8/Format-Serving/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 8991...
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:tensorflow:global_step/sec: 189.059
INFO:tensorflow:loss = 0.0010993367, step = 9000 (0.529 

ExecutionResult(
    component_id: Trainer
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 9
        type_id: 18
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Trainer/model/8"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 10
        type_id: 19
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Trainer/model_run/8"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 19
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [14]:
model_resolver = resolver.Resolver(
  strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
  model=Channel(type=Model),
  model_blessing=Channel(
      type=ModelBlessing)).with_id('latest_blessed_model_resolver')
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 9
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
eval_config = tfma.EvalConfig(
  model_specs=[tfma.ModelSpec(signature_name='eval')],
  slicing_specs=[
      tfma.SlicingSpec(),
      tfma.SlicingSpec(feature_keys=['trip_start_hour'])
  ],
  metrics_specs=[
      tfma.MetricsSpec(
          thresholds={
              'accuracy':
                  tfma.config.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.6}),
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10}))
          })
  ])

In [16]:
evaluator = Evaluator(
  examples=example_gen.outputs['examples'],
  model=trainer.outputs['model'],
  baseline_model=model_resolver.outputs['model'],
  eval_config=eval_config)
context.run(evaluator)

ERROR:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "thresholds": {\n        "accuracy": {\n          "change_threshold": {\n            "absolute": -1e-10,\n            "direction": "HIGHER_IS_BETTER"\n          },\n          "value_threshold": {\n            "lower_bound": 0.6\n          }\n        }\n      }\n    }\n  ],\n  "model_specs": [\n    {\n      "signature_name": "eval"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "trip_start_hour"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': None, 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_eval_shared_model'
ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).
ERROR:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "thresholds": {\n        "accuracy": {\n          "change_threshold": {\n           

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Trainer/model/8/Format-TFMA/variables/variables
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 10
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 11
        type_id: 21
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Evaluator/evaluation/10"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 12
        type_id: 22
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Evaluator/blessing/10"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Trainer/model/8"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 9
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
pusher = Pusher(
  model=trainer.outputs['model'],
  model_blessing=evaluator.outputs['blessing'],
  push_destination=pusher_pb2.PushDestination(
      filesystem=pusher_pb2.PushDestination.Filesystem(
          base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 11
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 13
        type_id: 24
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-07-23T15_02_28.164861-i4ijac0v/Pusher/pushed_model/11"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "serving_model/sampling_credit_card/1627067073"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1627067073"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))